<a href="https://colab.research.google.com/github/oldaandozerskaya/age_based_classification/blob/master/baseline_previews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

#Accumulated frequencies

The code used for calcutating accumulated frequencies to find the most informative features

In [ ]:
#accumulated frequencies

def acc_freq(mas):
  masmax=max(mas)
  masmin=min(mas)
  n=10
  part=(masmax-masmin)/float(n)
  borders=np.zeros(n)
  borders[0]=masmin+part
  borders[1]=borders[0]+part
  borders[2]=borders[1]+part
  borders[3]=borders[2]+part
  borders[4]=masmax

  frequences=np.zeros(n)
  for i in range(len(mas)):
    for j, bord in enumerate(borders):
      if mas[i]<=bord:
        frequences[j]+=1
        break

  for i in range(len(frequences)):
    frequences[i]=frequences[i]/float(len(mas))
  
  return frequences

def acc(f1):
  for i in range(1, len(f1)):
    f1[i]=f1[i-1]+f1[i]
  return f1

import math

def max_razn(f1, f2):
  razn=[]
  for i in range(len(f1)):
    razn.append(math.fabs(f1[i]-f2[i]))
  return max(razn)

coefs=[]
for j in range(additional_train.shape[1]):
  mas1 = df.iloc[:,j].values
  mas_ad=[]
  mas_ch=[]
  for z in range(len(train_labels)):
    if train_labels[z]==0:
      mas_ch.append(mas1[z])
    else:
      mas_ad.append(mas1[z])
  freq1=acc(acc_freq(mas_ch))
  freq2=acc(acc_freq(mas_ad))
  print(feature_names[j])
  print(freq1)
  print(freq2)
  coefs.append(max_razn(freq1, freq2))

In [ ]:
#print acc freq with feature names

for i in range(len(coefs)):
  print(str(coefs[i])+' - ' +str(feature_names[i]))

In [ ]:
#sorting frequencies
nums_features = np.array(coefs).argsort()[-16:][::-1]

In [ ]:
#top features

for num in nums_features:
  print(feature_names[num])
  print(coefs[num])
  mas1 = df.iloc[:,num].values
  mas_ad=[]
  mas_ch=[]
  for z in range(len(train_labels)):
    if train_labels[z]==0:
      mas_ch.append(mas1[z])
    else:
      mas_ad.append(mas1[z])
  print('children')
  print(np.mean(np.array(mas_ch)))
  print(np.std(np.array(mas_ch)))
  print('adults')
  print(np.mean(np.array(mas_ad)))
  print(np.std(np.array(mas_ad)))
  print('***')

#Baseline models 

The fragments of code related to the baseline models (RF & LSVC)

In [ ]:
#code for loading additional features

df=pd.read_csv('/content/drive/My Drive/children_and_adults/readability_train.csv', sep=';', dtype='unicode', index_col=0)
additional_train=df.values[1:]
print(additional_train.shape)

df=pd.read_csv('/content/drive/My Drive/children_and_adults/readability_test.csv', sep=';', index_col=0)
additional_test=df.values[1:]
print(additional_test.shape)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
additional_train = scaler.fit_transform(additional_train)
additional_test = scaler.transform(additional_test)

In [ ]:
PATH_TRAIN = '/content/drive/My Drive/children_and_adults/train/'
PATH_TEST = '/content/drive/My Drive/children_and_adults/test/'

In [ ]:
#tfidf for abstracts
from google.colab import drive
drive.mount('/content/drive')

import os

path_train='/content/drive/My Drive/children_and_adults/corpus/train'
path_test='/content/drive/My Drive/children_and_adults/corpus/test'
path_abstracts='/content/drive/My Drive/children_and_adults/corpus/abstracts'

In [ ]:
#adding abstracts

files_previews=os.listdir(path_train)
abstracts_train=[]

for i,f in enumerate(files_previews):
  with open(path_abstracts+'/'+f, 'r') as f42:
    abstracts_train.append(f42.read())

files_previews=os.listdir(path_test)
abstracts_test=[]

for i,f in enumerate(files_previews):
  with open(path_abstracts+'/'+f, 'r') as f42:
    abstracts_test.append(f42.read())

In [ ]:
#adding abstracts

!pip install pymorphy2
import pymorphy2, re
ma = pymorphy2.MorphAnalyzer()

def clean_text(text):
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols 
    text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    text = ' '.join(word for word in text.split() if len(word)>3)
    return text

abstracts_train_cleaned=[]
abstracts_test_cleaned=[]

for ab in abstracts_train:
  abstracts_train_cleaned.append(clean_text(ab))
for ab in abstracts_test:
  abstracts_test_cleaned.append(clean_text(ab))

In [ ]:
import pickle

with open('/content/drive/My Drive/children_and_adults/corpus/train_texts_cleaned_short.pickle', 'rb') as handle:
    train_texts=pickle.load(handle)
with open('/content/drive/My Drive/children_and_adults/corpus/test_texts_cleaned_short.pickle', 'rb') as handle:
    test_texts=pickle.load(handle)
with open('/content/drive/My Drive/children_and_adults/corpus/train_labels2.pickle', 'rb') as handle:
    train_labels=pickle.load(handle)
with open('/content/drive/My Drive/children_and_adults/corpus/val_labels2.pickle', 'rb') as handle:
    test_labels=pickle.load(handle)
print(len(train_texts))
print(len(test_texts))

In [ ]:
#adding all features

import pandas as pd
import numpy as np

df=pd.read_csv('/content/drive/My Drive/children_and_adults/rutez_train.csv', sep=';', index_col=0)
X=df.values
print(X.shape)

df=pd.read_csv('/content/drive/My Drive/children_and_adults/readability_train.csv', sep=';', index_col=0)
additional_train=df.values
print(additional_train.shape)
X=np.hstack((X,additional_train))

df=pd.read_csv('/content/drive/My Drive/children_and_adults/general_train.csv', sep=';', index_col=0)
additional_train=df.values
print(additional_train.shape)
X=np.hstack((X,additional_train))

df=pd.read_csv('/content/drive/My Drive/children_and_adults/gram_train.csv', sep=';', index_col=0)
additional_train=df.values
print(additional_train.shape)
X=np.hstack((X,additional_train))

df=pd.read_csv('/content/drive/My Drive/children_and_adults/lexical_train.csv', sep=';', index_col=0)
additional_train=df.values
print(additional_train.shape)
X=np.hstack((X,additional_train))

df=pd.read_csv('/content/drive/My Drive/children_and_adults/age_rating_train.csv', sep=';', index_col=0)
additional_train=df.values
print(additional_train.shape)
X=np.hstack((X,additional_train))

print(X.shape)
df=pd.DataFrame(X)
df.to_csv('all_features_train.csv', sep=';', header=0, index=None)

In [ ]:
df=pd.read_csv('all_features.csv', sep=';')
df.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(train_texts)
X1 = vectorizer.transform(test_texts)
#X = vectorizer.fit_transform(abstracts_train_cleaned)
#X1 = vectorizer.transform(abstracts_test_cleaned)

In [ ]:
X.shape

In [ ]:
import numpy as np

X=np.hstack((X.todense(),additional_train))
X1=np.hstack((X1.todense(),additional_test))
#X=np.hstack((X,additional_train))
#X1=np.hstack((X1,additional_test))

In [ ]:
X=additional_train
X1=additional_test

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
#Random Forest
best_res=0

f1=0
acc=0
pres=0
rec=0

for rs in range(10):
  RF=RandomForestClassifier(random_state=rs)
  RF.fit(X, train_labels)
  res=RF.predict(X1)
  f1=f1_score(res, test_labels)*100
  if f1>best_res:
    best_res=f1
    acc=accuracy_score(res, test_labels)*100
    rec=recall_score(res, test_labels)*100
    pres=precision_score(res, test_labels)*100

print('Accuracy: %f' % acc)
print('F1-score: %f' % best_res)
print('Precision: %f' % pres)
print('Recall: %f' % rec)

#LInear SVC

best_res=0

f1=0
acc=0
pres=0
rec=0

for rs in range(10):
  SVM=SVC(random_state=rs, tol=1e-5)
  SVM.fit(X, train_labels)
  res=SVM.predict(X1)
  f1=f1_score(res, test_labels)*100
  if f1>best_res:
      best_res=f1
      acc=accuracy_score(res, test_labels)*100
      rec=recall_score(res, test_labels)*100
      pres=precision_score(res, test_labels)*100

print('Accuracy: %f' % acc)
print('F1-score: %f' % best_res)
print('Precision: %f' % pres)
print('Recall: %f' % rec)

import numpy as np
from sklearn.decomposition import PCA
pca = PCA(.95)
try:
  X = pca.fit_transform(X.todense())
  X1 = pca.transform(X1.todense())
except:
  X = pca.fit_transform(X)
  X1 = pca.transform(X1) 
len(X[0])

best_res=0

f1=0
acc=0
pres=0
rec=0

for rs in range(10):
  SVM=SVC(random_state=rs, tol=1e-5)
  SVM.fit(X, train_labels)
  res=SVM.predict(X1)
  f1=f1_score(res, test_labels)*100
  if f1>best_res:
      best_res=f1
      acc=accuracy_score(res, test_labels)*100
      rec=recall_score(res, test_labels)*100
      pres=precision_score(res, test_labels)*100

print('Accuracy: %f' % acc)
print('F1-score: %f' % best_res)
print('Precision: %f' % pres)
print('Recall: %f' % rec)

In [ ]:

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train_texts)
X1 = vectorizer.transform(test_texts)

#Random Forest

RF=RandomForestClassifier(random_state=0)
RF.fit(X, train_labels)
res=RF.predict(X1)
print('Accuracy: %f' % (accuracy_score(res, test_labels)*100))
print('F1-score: %f' % (f1_score(res, test_labels)*100))
print('Precision: %f' % (precision_score(res, test_labels)*100))
print('Recall: %f' % (recall_score(res, test_labels)*100))

#LInear SVC

SVM=LinearSVC(random_state=0, tol=1e-5)
SVM.fit(X, train_labels)
res=SVM.predict(X1)
print('Accuracy: %f' % (accuracy_score(res, test_labels)*100))
print('F1-score: %f' % (f1_score(res, test_labels)*100))
print('Precision: %f' % (precision_score(res, test_labels)*100))
print('Recall: %f' % (recall_score(res, test_labels)*100))

In [ ]:
####readability

import requests
import nltk
from nltk import word_tokenize, sent_tokenize

def average_sentence_length(text):
  sentences=sent_tokenize(text)
  l=0
  for sent in sentences:
    l+=len(word_tokenize(sent))
  return l/len(sentences)

def number_of_syllables(word):
  count=0
  word=word.lower()
  for w in word:
    if w=='а' or w=='ё' or w=='е' or w=='у' or w=='э' or w=='ю' or w=='и' or w=='я' or w=='ы' or w=='о':
        count+=1
  return count

def average_syllabeles_word(text):
  words=word_tokenize(text)
  l=0
  for word in words:
    l+=number_of_syllables(word)
  return l/len(words)

values=[]
values.append(['index_fk', 'index_cl', 'index_dc', 'index_ari', 'index_SMOG'])

for text in train_texts:
  response = requests.post("http://api.plainrussian.ru/api/1.0/ru/measure/", data={"text":text})
  res=response.json()
  values.append([res['indexes']['index_fk'], res['indexes']['index_cl'], res['indexes']['index_dc'], res['indexes']['index_ari'], res['indexes']['index_SMOG']])


In [ ]:
df=pd.DataFrame(values)
df[:10]

In [ ]:
df.to_csv('readability_train.csv', sep=';', inde)

In [ ]:
with open('/content/drive/My Drive/children_and_adults/5000lemma.al', 'r', encoding='cp1251') as handle:
  content=handle.readlines()

words=[]
freq=[]
for i in range(1,len(content)):
  elems=content[i].split()
  words.append(elems[2])
  freq.append(elems[1])

dictionary = dict(zip(words, freq))

In [ ]:
path_freq='/content/drive/My Drive/freqrnc2011.csv'
df=pd.read_csv(path_freq, sep='\t')
df

In [ ]:
list_dict=df['Lemma'].values
pos_dict=df['PoS'].values
freq_dict=df['Freq(ipm)'].values
r_dict=df['R'].values
d_dict=df['D'].values
doc_dict=df['Doc'].values

values_lex=[]
values_lex.append(['words_fr', 'words_r', 'words_d', 'words_doc', 'v_fr', 'v_r', 'v_d', 'v_doc',\
            's_fr', 's_r', 's_d', 's_doc', 'adv_fr', 'adv_r', 'adv_d', 'adv_doc',\
            'adj_fr', 'adj_r', 'adj_d', 'adj_doc', 'prop_fr', 'prop_r', 'prop_d', 'prop_doc', '5000_proc', '5000_freq' ])
values_gram=[]
values_gram.append(['count_v', 'count_s', 'count_adj'])

list_dict=[w.lower() for w in list_dict]

for t in train_texts:
  words=t.split()

  words_in_list=0
  verbs_in_list=0
  nouns_in_list=0
  adv_in_list=0
  adj_in_list=0
  propn_in_list=0

  words_freq=0
  words_r=0
  words_d=0
  words_doc=0

  v_freq=0
  v_r=0
  v_d=0
  v_doc=0

  adv_freq=0
  adv_r=0
  adv_d=0
  adv_doc=0

  s_freq=0
  s_r=0
  s_d=0
  s_doc=0

  a_freq=0
  a_r=0
  a_d=0
  a_doc=0

  p_freq=0
  p_r=0
  p_d=0
  p_doc=0

  number_words=len(words)
  number_in_list=0
  number_out_of_list=0
  freq=0

  for word in words:
    try:
      freq=dictionary[word]
      number_in_list+=1
    except:
      number_out_of_list+=1

    for i, item in enumerate(list_dict):
      if word==item:
        words_in_list+=1
        words_freq+=freq_dict[i]
        words_r+=r_dict[i]
        words_d+=d_dict[i]
        words_doc+=doc_dict[i]

        if pos_dict[i]=='v':
          verbs_in_list+=1
          v_freq+=freq_dict[i]
          v_r+=r_dict[i]
          v_d+=d_dict[i]
          v_doc+=doc_dict[i]
        elif pos_dict[i]=='adv' or pos_dict[i]=='advpro':
          adv_in_list+=1
          adv_freq+=freq_dict[i]
          adv_r+=r_dict[i]
          adv_d+=d_dict[i]
          adv_doc+=doc_dict[i]
        elif pos_dict[i]=='s':
          nouns_in_list+=1
          s_freq+=freq_dict[i]
          s_r+=r_dict[i]
          s_d+=d_dict[i]
          s_doc+=doc_dict[i]
        elif pos_dict[i]=='a':
          adj_in_list+=1
          a_freq+=freq_dict[i]
          a_r+=r_dict[i]
          a_d+=d_dict[i]
          a_doc+=doc_dict[i]
        elif pos_dict[i]=='s.PROP':
          propn_in_list+=1
          p_freq+=freq_dict[i]
          p_r+=r_dict[i]
          p_d+=d_dict[i]
          p_doc+=doc_dict[i]

  new_line=[]
  if words_in_list!=0:
    new_line.append(words_freq/words_in_list)
    new_line.append(words_r/words_in_list)
    new_line.append(words_d/words_in_list)
    new_line.append(words_doc/words_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if verbs_in_list!=0:
    new_line.append(v_freq/verbs_in_list)
    new_line.append(v_r/verbs_in_list)
    new_line.append(v_d/verbs_in_list)
    new_line.append(v_doc/verbs_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if nouns_in_list!=0:
    new_line.append(s_freq/nouns_in_list)
    new_line.append(s_r/nouns_in_list)
    new_line.append(s_d/nouns_in_list)
    new_line.append(s_doc/nouns_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
  
  if adv_in_list!=0:
    new_line.append(adv_freq/adv_in_list)
    new_line.append(adv_r/adv_in_list)
    new_line.append(adv_d/adv_in_list)
    new_line.append(adv_doc/adv_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if adj_in_list!=0:
    new_line.append(a_freq/adj_in_list)
    new_line.append(a_r/adj_in_list)
    new_line.append(a_d/adj_in_list)
    new_line.append(a_doc/adj_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if propn_in_list!=0:
    new_line.append(p_freq/propn_in_list)
    new_line.append(p_r/propn_in_list)
    new_line.append(p_d/propn_in_list)
    new_line.append(p_doc/propn_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
  
  if number_words!=0:
    new_line.append(float(number_in_list)/number_words)
  else:
    new_line.append(0)
  
  if number_in_list!=0:
    new_line.append(float(freq)/number_in_list)
  else:
    new_line.append(0)

  values_lex.append(new_line)
  new_line=[float(verbs_in_list)/len(words), float(nouns_in_list)/len(words), float(adj_in_list)/len(words)]
  values_gram.append(new_line)

df=pd.DataFrame(values_lex)
df.to_csv('/content/drive/My Drive/children_and_adults/lexical_train.csv', sep=';')
df=pd.DataFrame(values_gram)
df.to_csv('/content/drive/My Drive/children_and_adults/gram_train.csv', sep=';')

In [ ]:
list_dict=df['Lemma'].values
pos_dict=df['PoS'].values
freq_dict=df['Freq(ipm)'].values
r_dict=df['R'].values
d_dict=df['D'].values
doc_dict=df['Doc'].values

values_lex=[]
values_lex.append(['words_fr', 'words_r', 'words_d', 'words_doc', 'v_fr', 'v_r', 'v_d', 'v_doc',\
            's_fr', 's_r', 's_d', 's_doc', 'adv_fr', 'adv_r', 'adv_d', 'adv_doc',\
            'adj_fr', 'adj_r', 'adj_d', 'adj_doc', 'prop_fr', 'prop_r', 'prop_d', 'prop_doc', '5000_proc', '5000_freq' ])
values_gram=[]
values_gram.append(['count_v', 'count_s', 'count_adj'])

list_dict=[w.lower() for w in list_dict]

for t in test_texts:
  words=t.split()

  words_in_list=0
  verbs_in_list=0
  nouns_in_list=0
  adv_in_list=0
  adj_in_list=0
  propn_in_list=0

  words_freq=0
  words_r=0
  words_d=0
  words_doc=0

  v_freq=0
  v_r=0
  v_d=0
  v_doc=0

  adv_freq=0
  adv_r=0
  adv_d=0
  adv_doc=0

  s_freq=0
  s_r=0
  s_d=0
  s_doc=0

  a_freq=0
  a_r=0
  a_d=0
  a_doc=0

  p_freq=0
  p_r=0
  p_d=0
  p_doc=0

  number_words=len(words)
  number_in_list=0
  number_out_of_list=0
  freq=0

  for word in words:
    try:
      freq=dictionary[word]
      number_in_list+=1
    except:
      number_out_of_list+=1

    for i, item in enumerate(list_dict):
      if word==item:
        words_in_list+=1
        words_freq+=freq_dict[i]
        words_r+=r_dict[i]
        words_d+=d_dict[i]
        words_doc+=doc_dict[i]

        if pos_dict[i]=='v':
          verbs_in_list+=1
          v_freq+=freq_dict[i]
          v_r+=r_dict[i]
          v_d+=d_dict[i]
          v_doc+=doc_dict[i]
        elif pos_dict[i]=='adv' or pos_dict[i]=='advpro':
          adv_in_list+=1
          adv_freq+=freq_dict[i]
          adv_r+=r_dict[i]
          adv_d+=d_dict[i]
          adv_doc+=doc_dict[i]
        elif pos_dict[i]=='s':
          nouns_in_list+=1
          s_freq+=freq_dict[i]
          s_r+=r_dict[i]
          s_d+=d_dict[i]
          s_doc+=doc_dict[i]
        elif pos_dict[i]=='a':
          adj_in_list+=1
          a_freq+=freq_dict[i]
          a_r+=r_dict[i]
          a_d+=d_dict[i]
          a_doc+=doc_dict[i]
        elif pos_dict[i]=='s.PROP':
          propn_in_list+=1
          p_freq+=freq_dict[i]
          p_r+=r_dict[i]
          p_d+=d_dict[i]
          p_doc+=doc_dict[i]

  new_line=[]
  if words_in_list!=0:
    new_line.append(words_freq/words_in_list)
    new_line.append(words_r/words_in_list)
    new_line.append(words_d/words_in_list)
    new_line.append(words_doc/words_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if verbs_in_list!=0:
    new_line.append(v_freq/verbs_in_list)
    new_line.append(v_r/verbs_in_list)
    new_line.append(v_d/verbs_in_list)
    new_line.append(v_doc/verbs_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if nouns_in_list!=0:
    new_line.append(s_freq/nouns_in_list)
    new_line.append(s_r/nouns_in_list)
    new_line.append(s_d/nouns_in_list)
    new_line.append(s_doc/nouns_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
  
  if adv_in_list!=0:
    new_line.append(adv_freq/adv_in_list)
    new_line.append(adv_r/adv_in_list)
    new_line.append(adv_d/adv_in_list)
    new_line.append(adv_doc/adv_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if adj_in_list!=0:
    new_line.append(a_freq/adj_in_list)
    new_line.append(a_r/adj_in_list)
    new_line.append(a_d/adj_in_list)
    new_line.append(a_doc/adj_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)

  if propn_in_list!=0:
    new_line.append(p_freq/propn_in_list)
    new_line.append(p_r/propn_in_list)
    new_line.append(p_d/propn_in_list)
    new_line.append(p_doc/propn_in_list)
  else:
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
    new_line.append(0)
  
  if number_words!=0:
    new_line.append(float(number_in_list)/number_words)
  else:
    new_line.append(0)
  
  if number_in_list!=0:
    new_line.append(float(freq)/number_in_list)
  else:
    new_line.append(0)

  values_lex.append(new_line)
  new_line=[float(verbs_in_list)/len(words), float(nouns_in_list)/len(words), float(adj_in_list)/len(words)]
  values_gram.append(new_line)

df=pd.DataFrame(values_lex)
df.to_csv('/content/drive/My Drive/children_and_adults/lexical_test.csv', sep=';')
df=pd.DataFrame(values_gram)
df.to_csv('/content/drive/My Drive/children_and_adults/gram_test.csv', sep=';')

In [ ]:
for i,p in enumerate(df['PoS'].values):
  if p=='pr':
    print(df['Lemma'].values[i])


In [ ]:
###ЧАСТОТНЫЙ СЛОВАРЬ

values = []
values.append([])

for t in test_texts:
  text_words=t.split()
  number_words=len(text_words)
  number_in_list=0
  number_out_of_list=0
  freq=0
  for w in text_words:
    try:
      freq=dictionary[w]
      number_in_list+=1
    except:
      number_out_of_list+=1

  try:
    values.append([float(number_in_list)/number_words, float(freq)/number_in_list])
  except:
    values.append([0, 0])
df=pd.DataFrame(values)
df.to_csv('/content/drive/My Drive/children_and_adults/lexmin_test.csv', sep=';')

In [ ]:
#age rating
import os

PATH_DESCRIPTIONS = '/content/drive/My Drive/children_and_adults/description.csv'
df=pd.read_csv(PATH_DESCRIPTIONS, sep=';', header=None)

dictionary = dict(zip(df[0], df[3]))

PATH_TRAIN = '/content/drive/My Drive/children_and_adults/train/'
PATH_TEST = '/content/drive/My Drive/children_and_adults/test/'

files=os.listdir(PATH_TRAIN)
print(len(files))
ratings=[]
ratings.append(['age_rating'])

for f in files:
  rating=dictionary[str(f)]
  if rating==0:
    rating.append([0])
  elif rating==6:
    ratings.append([0.25])
  elif rating==12:
    ratings.append([0.5])
  elif rating==16:
    ratings.append([0.75])
  elif rating==18:
    ratings.append([1])
  else:
    ratings.append([0.5])
df=pd.DataFrame(ratings)
df.to_csv('/content/drive/My Drive/children_and_adults/age_rating_train.csv', sep=';')

files=os.listdir(PATH_TEST)
ratings=[]
ratings.append(['age_rating'])

for f in files:
  rating=dictionary[str(f)]
  if rating==0:
    rating.append([0])
  elif rating==6:
    ratings.append([0.25])
  elif rating==12:
    ratings.append([0.5])
  elif rating==16:
    ratings.append([0.75])
  elif rating==18:
    ratings.append([1])
  else:
    ratings.append([0.5])
df=pd.DataFrame(ratings)
df.to_csv('/content/drive/My Drive/children_and_adults/age_rating_test.csv', sep=';')